# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd

file_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

data = pd.read_csv(file_url)

In [2]:
data.columns = (
    data.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)


In [3]:
filtered_data = data[(data['total_claim_amount'] > 1000) & (data['response'] == 'Yes')]

In [4]:
print(filtered_data.shape[0])

67


In [5]:
filtered_data.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


In [6]:
responded_yes = data[data['response'] == 'Yes']

In [7]:
average_claim = responded_yes.pivot_table(
    values='total_claim_amount',  # Columna de la que queremos el promedio
    index='policy_type',          # Filas: Tipo de póliza
    columns='gender',             # Columnas: Género
    aggfunc='mean'                # Agregación: Promedio
)
average_claim = average_claim.round(2)

In [8]:
print(average_claim)

gender               F       M
policy_type                   
Corporate Auto  433.74  408.58
Personal Auto   452.97  457.01
Special Auto    453.28  429.53


In [10]:
customers_by_state = data['state'].value_counts()
states_with_more_than_500 = customers_by_state[customers_by_state > 500]

In [12]:
print(states_with_more_than_500)

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [15]:
clv_stats = data.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()
print(clv_stats)

              education gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [16]:
data['effective_to_date'] = pd.to_datetime(data['effective_to_date'])

/var/folders/9b/1k3cjn9x4cg_w6793ryzh2hw0000gn/T/ipykernel_77586/3360401548.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['effective_to_date'] = pd.to_datetime(data['effective_to_date'])


In [17]:
data['month'] = data['effective_to_date'].dt.month

In [18]:
policies_by_state_month = data.pivot_table(
    index='state',          # Fila: Estado
    columns='month',        # Columna: Mes
    values='policy',        # Conteo basado en la columna 'policy' (puede ser cualquier columna no vacía)
    aggfunc='count',        # Función de agregación: contar
    fill_value=0            # Reemplazar NaN por 0
)
print(policies_by_state_month)

month          1     2
state                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


In [20]:
data = pd.read_csv(file_url)
data.columns = (
    data.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

In [21]:
data['effective_to_date'] = pd.to_datetime(data['effective_to_date'])
data['month'] = data['effective_to_date'].dt.month

total_policies_by_state = data['state'].value_counts()

top_3_states = total_policies_by_state.head(3).index

filtered_data = data[data['state'].isin(top_3_states)]

/var/folders/9b/1k3cjn9x4cg_w6793ryzh2hw0000gn/T/ipykernel_77586/470678941.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['effective_to_date'] = pd.to_datetime(data['effective_to_date'])


In [22]:
top_3_policies_by_month = filtered_data.pivot_table(
    index='state',         # Fila: Estado
    columns='month',       # Columna: Mes
    values='policy',       # Conteo basado en la columna 'policy' (o cualquier columna no vacía)
    aggfunc='count',       # Función de agregación: contar
    fill_value=0           # Reemplazar NaN por 0
)
print(top_3_policies_by_month)


month          1     2
state                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344
